In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.utils import *
from utils.kabsch2D import *
from nsd_access import NSDAccess 
from scipy import stats
from utils.flips import create_rotation_df, get_ranking, get_rank_dict
from utils.stats import rank_correlation, parametric_test, wilcoxon_test
from scipy.spatial.distance import pdist

In [210]:
df = pd.read_csv('rotations/subj01_rotations_df.csv')
df

,Unnamed: 0,source,base,target,U,distance
0,0,LO-1,VO-1,VO-1_flipped,[[ 0.74715103 0.6646543 ]\n [-0.6646543 0.7...,0.744939
1,1,LO-2,VO-1,VO-1,[[ 0.5988554 -0.8008572]\n [ 0.8008572 0.5988...,0.808128
2,2,PHC-1,VO-1,VO-1,[[-0.68899155 -0.72476935]\n [ 0.72476935 -0.6...,0.588861
3,3,PHC-2,VO-1,VO-1_flipped,[[ 0.85219646 -0.52322194]\n [ 0.52322194 0.8...,0.730155
4,4,TO-1,VO-1,VO-1_flipped,[[-0.64399133 -0.76503279]\n [ 0.76503279 -0.6...,0.844953
5,5,TO-2,VO-1,VO-1,[[-0.62602067 0.7798064 ]\n [-0.7798064 -0.6...,0.750447
6,6,V1,VO-1,VO-1,[[-0.9546826 -0.29762578]\n [ 0.29762578 -0.9...,0.577720
7,7,V2,VO-1,VO-1,[[ 0.23635045 0.9716679 ]\n [-0.9716679 0.2...,0.696862
8,8,V3,VO-1,VO-1,[[-0.6324826 -0.77457464]\n [ 0.77457464 -0.6...,0.485322
9,9,VO-2,VO-1,VO-1,[[-0.43945456 0.8982649 ]\n [-0.8982649 -0.4...,0.635558


In [220]:
if "flipped" in df[df["source"]== "LO-1"].target[0]:
    print(True)

True


In [221]:
arr = np.arange(10)
indexes = np.array([1, 2])
arr[indexes] = 0


array([ 0,  0,  0, 15, 20, 25, 30, 35, 40, 45])

In [83]:
import nibabel as nib
file = os.path.join(results_dir, 'subj01', 'lh.oself.var_explained.notrotated.mgz')
img = nib.loadsave.load(file)
img.dataobj.get_unscaled().flatten()

# I know the issue: 0 is outside the mask (so outside the visual system) 
# meaning we have negative fits, 0s and positive fits 
# tresholding right above 0 then should make sense: we remove the negative fits and the unwanted voxel 

array([1.0400262, 1.0095049, 1.0323589, ..., 0.       , 0.       ,
       0.       ], dtype='>f4')

In [73]:
import nibabel as nib
file = os.path.join(results_dir, 'subj01', 'lh.oself.mds_ecc.rotated.mgz')
img = nib.loadsave.load(file)
img.dataobj.get_unscaled().flatten()[:30]

array([0.5239681 , 0.557804  , 0.62293994, 0.5080766 , 0.49640885,
       0.54625475, 0.4737816 , 0.49423653, 0.87378174, 0.8665644 ,
       0.7462431 , 0.7975724 , 0.7364943 , 0.65769035, 0.76570296,
       0.707056  , 0.58152986, 0.51834404, 0.41586563, 0.808049  ,
       0.5235812 , 0.8909445 , 0.44309577, 0.44051215, 0.47157997,
       0.5168668 , 0.51866883, 0.50638497, 0.51131624, 0.4504067 ],
      dtype='>f4')

In [74]:
import nibabel as nib
file = os.path.join(results_dir, 'subj01', 'lh.oself.mds_ecc.notrotated.mgz')
img = nib.loadsave.load(file)
img.dataobj.get_unscaled().flatten()[:30]

array([1.113759 , 1.0671877, 1.1640403, 1.1065581, 1.1012381, 1.124514 ,
       1.0912812, 1.1003652, 1.2756858, 1.3116245, 1.2159837, 1.2740506,
       1.242086 , 1.1741755, 1.2376608, 1.2239627, 1.175378 , 1.1180112,
       1.1767066, 1.3154688, 1.1136215, 1.3612069, 1.0782964, 1.0584064,
       1.0902623, 1.11088  , 1.1113036, 1.1060175, 1.1081369, 1.0812299],
      dtype='>f4')

In [75]:
import nibabel as nib
file = os.path.join(results_dir, 'subj02', 'lh.oself.mds_ecc.notrotated.mgz')
img = nib.loadsave.load(file)
img.dataobj.get_unscaled().flatten()[:30]

array([1.4849242, 1.484924 , 1.4849234, 1.4849242, 1.4849242, 1.4849242,
       1.4849242, 1.4849242, 1.4849223, 1.4849242, 1.4849242, 1.4849242,
       1.4849232, 1.052747 , 1.4552053, 1.4849242, 1.4849242, 1.0511075,
       1.4849242, 1.4849242, 1.4849226, 1.4849098, 1.4848903, 1.4849235,
       1.4849242, 1.4849242, 1.4849242, 1.484899 , 1.484862 , 1.4844418],
      dtype='>f4')

In [77]:
import nibabel as nib
file = os.path.join(results_dir, 'subj03', 'lh.oself.mds_ecc.notrotated.mgz')
img = nib.loadsave.load(file)
img.dataobj.get_unscaled().flatten()[:30]

# I think the issue here is that they all operate in different scales
# I can normalize these values? 

array([1.0841323 , 1.054514  , 1.0526019 , 1.052084  , 1.0737656 ,
       1.0549861 , 1.0559356 , 1.057789  , 0.44487882, 1.1175268 ,
       1.0808185 , 1.0729269 , 0.40633702, 0.41941348, 1.069972  ,
       1.0634967 , 1.0552471 , 0.40474576, 1.0609215 , 1.0649314 ,
       1.0596117 , 1.0614363 , 1.0834934 , 1.0693158 , 1.0780811 ,
       1.0631019 , 0.17811768, 0.45413038, 1.163476  , 1.1055461 ],
      dtype='>f4')

In [5]:
def all_subj_rank(dict, rois, random=True):
    """
    Take the dictionary containing the ranking for all participants and 
    return one final ranking, that is simply the sum of all rankings
    """
    if random:
        rank_total = np.zeros(len(rois) + 1)
    else:
        rank_total = np.zeros(len(rois))
    for i in range(len(dict)):
        rank_total += list(dict.values())[i]
    return rank_total

In [60]:
a = np.random.rand(5, 5)
a[0, 1] = np.nan
a[3, 4] = np.nan
print(a)
a = np.nan_to_num(a, copy=False)
a

[[0.99522628        nan 0.87615979 0.90210823 0.05242401]
 [0.13829398 0.94572516 0.83524315 0.70022098 0.36170967]
 [0.30204471 0.3001503  0.70357014 0.52567658 0.98880789]
 [0.06357905 0.73662866 0.2963342  0.9886945         nan]
 [0.35289217 0.45856062 0.80299283 0.03757054 0.43120569]]


array([[0.99522628, 0.        , 0.87615979, 0.90210823, 0.05242401],
       [0.13829398, 0.94572516, 0.83524315, 0.70022098, 0.36170967],
       [0.30204471, 0.3001503 , 0.70357014, 0.52567658, 0.98880789],
       [0.06357905, 0.73662866, 0.2963342 , 0.9886945 , 0.        ],
       [0.35289217, 0.45856062, 0.80299283, 0.03757054, 0.43120569]])

In [7]:

rank_dict, rank_dict_values = get_rank_dict(subj_list, rois, random=random, mode='averaged')

In [8]:
rank_total = all_subj_rank(rank_dict_values, rois)
rank_total.sort_values()

source
VO-1      5.046467
VO-2      5.134792
V3        5.242516
PHC-1     5.270800
hV4       5.271054
LO-1      5.280855
PHC-2     5.289833
TO-2      5.310410
V1        5.395471
LO-2      5.399963
V2        5.461472
TO-1      5.597116
random    6.962392
Name: distance, dtype: float64

In [9]:
rank_dict_values_norm = {key : (value) / (value.max()) for key, value in rank_dict_values.items()} #normalize with random
print(rank_dict_values_norm)
rank_total_norm = all_subj_rank(rank_dict_values_norm, rois)
print(rank_total_norm.sort_values())

{'subj01': source
VO-2      0.755494
hV4       0.764712
PHC-2     0.765413
V2        0.769307
VO-1      0.775515
LO-1      0.783133
PHC-1     0.783489
V1        0.788727
V3        0.793096
TO-2      0.816848
LO-2      0.829340
TO-1      0.860395
random    1.000000
Name: distance, dtype: float32, 'subj02': source
VO-1      0.743994
LO-1      0.751448
TO-2      0.763778
PHC-2     0.775733
TO-1      0.781330
V3        0.784659
LO-2      0.784966
VO-2      0.787962
V1        0.788483
V2        0.809553
hV4       0.812576
PHC-1     0.858298
random    1.000000
Name: distance, dtype: float32, 'subj03': source
PHC-1     0.654636
VO-2      0.666067
VO-1      0.668514
TO-2      0.693667
hV4       0.701157
V3        0.705846
V1        0.720183
LO-1      0.722678
V2        0.742286
PHC-2     0.744012
LO-2      0.764055
TO-1      0.788509
random    1.000000
Name: distance, dtype: float32, 'subj04': source
PHC-1     0.703211
PHC-2     0.706779
LO-1      0.726287
LO-2      0.733191
VO-1      0.733865

## KENDALL TAU CORRELATION TEST

In [8]:
corr_df, corr_ps = rank_correlation(rank_dict)
corr_df

,subj01,subj02,subj03,subj04,subj05,subj06,subj07,subj08
subj01,1.000000,0.076923,0.205128,-0.153846,-0.128205,0.051282,0.743590,0.435897
subj02,0.076923,1.000000,0.153846,0.102564,0.435897,0.512821,0.076923,0.128205
subj03,0.205128,0.153846,1.000000,-0.025641,0.000000,-0.179487,0.205128,0.102564
subj04,-0.153846,0.102564,-0.025641,1.000000,0.102564,0.282051,-0.205128,-0.102564
subj05,-0.128205,0.435897,0.000000,0.102564,1.000000,0.307692,-0.128205,-0.076923
subj06,0.051282,0.512821,-0.179487,0.282051,0.307692,1.000000,-0.051282,0.256410
subj07,0.743590,0.076923,0.205128,-0.205128,-0.128205,-0.051282,1.000000,0.435897
subj08,0.435897,0.128205,0.102564,-0.102564,-0.076923,0.256410,0.435897,1.000000


In [9]:
corr_ps

,subj01,subj02,subj03,subj04,subj05,subj06,subj07,subj08
subj01,0.00000,0.76502,0.36740,0.50977,0.59004,0.85773,0.00015,0.04218
subj02,0.76502,0.00000,0.50977,0.67543,0.04218,0.01495,0.76502,0.59004
subj03,0.36740,0.50977,0.00000,0.95236,1.00000,0.43538,0.36740,0.67543
subj04,0.50977,0.67543,0.95236,0.00000,0.67543,0.20437,0.36740,0.67543
subj05,0.59004,0.04218,1.00000,0.67543,0.00000,0.16349,0.59004,0.76502
subj06,0.85773,0.01495,0.43538,0.20437,0.16349,0.00000,0.85773,0.25186
subj07,0.00015,0.76502,0.36740,0.36740,0.59004,0.85773,0.00000,0.04218
subj08,0.04218,0.59004,0.67543,0.67543,0.76502,0.25186,0.04218,0.00000


## PARAMETRIC TEST

In [11]:
parametric_test(subj_list, rois, iterations=100) # run for all participants, choose your iterations
# saved as a csv because it take ages to run 

KeyboardInterrupt: 

In [6]:
subj_01_param = os.path.join(param_dir, 'subj01_parametric_test_output.csv')
subj_01_param = pd.read_csv(subj_01_param)
subj_01_param

FileNotFoundError: [Errno 2] No such file or directory: '/media/Working/stan-thesis/projects/stats/parametric_test/subj01_parametric_test_output.csv'

In [ ]:
plt.hist(subj_01_param.V1)
plt.show()

## WILCOXON TEST 

In [10]:
z_values_df_W, p_values_df_W, median_df_W = wilcoxon_test(subj_list, rois)
z_values_df_W # check z values 

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
V1,0,-2.004848,0.964054,-0.935276,5.376638,-0.398092,1.678701,0.124703,-0.776999,-0.681073,-1.405312,1.064776
V2,2.004848,0,1.462868,1.860959,4.887417,3.093605,3.314234,1.515627,0.729036,1.064776,-0.474832,3.24229
V3,-0.964054,-1.462868,0,-0.455647,5.088861,0.757813,1.678701,0.177463,-0.776999,-0.657091,-1.414905,1.376534
hV4,0.935276,-1.860959,0.455647,0,2.935328,1.587571,1.51083,0.057555,-0.844147,-0.959257,-1.534812,0.940072
VO-1,-5.376638,-4.887417,-5.088861,-2.935328,0,-2.014441,-2.647551,-2.220681,-2.925735,-2.863383,-3.237494,-1.721867
VO-2,0.398092,-3.093605,-0.757813,-1.587571,2.014441,0,0.93048,-0.455647,-1.520423,-1.280609,-1.836978,0.350129
PHC-1,-1.678701,-3.314234,-1.678701,-1.51083,2.647551,-0.93048,0,-1.371738,-2.594791,-2.062403,-2.666736,-0.901702
PHC-2,-0.124703,-1.515627,-0.177463,-0.057555,2.220681,0.455647,1.371738,0,-0.767406,-1.347757,-3.386179,-0.537184
LO-1,0.776999,-0.729036,0.776999,0.844147,2.925735,1.520423,2.594791,0.767406,0,-0.575554,-3.060031,1.97607
LO-2,0.681073,-1.064776,0.657091,0.959257,2.863383,1.280609,2.062403,1.347757,0.575554,0,-3.985715,1.592367


In [11]:
median_df_W


,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
V1,0,-0.010804,0.010588,-0.011136,0.047163,-0.006585,0.020776,0.013245,-0.012745,-0.020736,-0.030389,0.026682
V2,0.010804,0,0.006282,0.002006,0.064068,0.022581,0.052663,0.033119,0.0191,0.01104,-0.014544,0.048168
V3,-0.010588,-0.006282,0,-0.014423,0.037562,0.001115,0.026541,0.002955,-0.009169,-0.006151,-0.029323,0.032216
hV4,0.011136,-0.002006,0.014423,0,0.050678,0.005452,0.034452,0.032846,-0.006557,-0.019903,-0.037464,0.015021
VO-1,-0.047163,-0.064068,-0.037562,-0.050678,0,-0.03383,-0.026947,-0.068785,-0.069363,-0.076832,-0.10407,-0.031269
VO-2,0.006585,-0.022581,-0.001115,-0.005452,0.03383,0,0.024254,-0.005066,-0.027823,-0.008873,-0.044483,0.016113
PHC-1,-0.020776,-0.052663,-0.026541,-0.034452,0.026947,-0.024254,0,-0.029827,-0.049055,-0.043061,-0.048181,-0.009799
PHC-2,-0.013245,-0.033119,-0.002955,-0.032846,0.068785,0.005066,0.029827,0,-0.000537,-0.00748,-0.02907,-0.00061
LO-1,0.012745,-0.0191,0.009169,0.006557,0.069363,0.027823,0.049055,0.000537,0,-0.008921,-0.026163,0.031106
LO-2,0.020736,-0.01104,0.006151,0.019903,0.076832,0.008873,0.043061,0.00748,0.008921,0,-0.021873,0.023568


In [12]:
p_values_df_W

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
V1,1,0.044979,0.335019,0.349646,0.0,0.690562,0.09321,0.900758,0.43716,0.495825,0.159928,0.286977
V2,0.044979,1,0.143504,0.06275,0.000001,0.001977,0.000919,0.129614,0.46598,0.286977,0.634906,0.001186
V3,0.335019,0.143504,1,0.648644,0.0,0.448563,0.09321,0.859145,0.43716,0.511122,0.157096,0.168656
hV4,0.349646,0.06275,0.648644,1,0.003332,0.112383,0.130832,0.954103,0.398588,0.337429,0.12483,0.34718
VO-1,0.0,0.000001,0.0,0.003332,1,0.043963,0.008108,0.026373,0.003436,0.004191,0.001206,0.085094
VO-2,0.690562,0.001977,0.448563,0.112383,0.043963,1,0.352123,0.648644,0.128405,0.200331,0.066213,0.726242
PHC-1,0.09321,0.000919,0.09321,0.130832,0.008108,0.352123,1,0.170145,0.009465,0.039169,0.007659,0.367215
PHC-2,0.900758,0.129614,0.859145,0.954103,0.026373,0.648644,0.170145,1,0.44284,0.177737,0.000709,0.59114
LO-1,0.43716,0.46598,0.43716,0.398588,0.003436,0.128405,0.009465,0.44284,1,0.564916,0.002213,0.048147
LO-2,0.495825,0.286977,0.511122,0.337429,0.004191,0.200331,0.039169,0.177737,0.564916,1,0.000067,0.111302


In [6]:
x=np.arange(1, 6.1, 0.1)
print(x)

[1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7
 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5 3.6 3.7 3.8 3.9 4.  4.1 4.2 4.3 4.4 4.5
 4.6 4.7 4.8 4.9 5.  5.1 5.2 5.3 5.4 5.5 5.6 5.7 5.8 5.9 6. ]


In [9]:
x=np.arange(1, 6.1, 0.1)
print(x)
res = stats.wilcoxon(x, method='approx')
res.zstatistic

[1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7
 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5 3.6 3.7 3.8 3.9 4.  4.1 4.2 4.3 4.4 4.5
 4.6 4.7 4.8 4.9 5.  5.1 5.2 5.3 5.4 5.5 5.6 5.7 5.8 5.9 6. ]


-6.214608533391381

In [8]:
median_df_W

NameError: name 'median_df_W' is not defined

In [7]:
x = np.arange(1, 6.1, 0.1)
res = stats.wilcoxon(x, method='approx')
print(res.zstatistic)

-6.214608533391381


In [15]:
-x

array([-1. , -1.1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7, -1.8, -1.9, -2. ,
       -2.1, -2.2, -2.3, -2.4, -2.5, -2.6, -2.7, -2.8, -2.9, -3. , -3.1,
       -3.2, -3.3, -3.4, -3.5, -3.6, -3.7, -3.8, -3.9, -4. , -4.1, -4.2,
       -4.3, -4.4, -4.5, -4.6, -4.7, -4.8, -4.9, -5. , -5.1, -5.2, -5.3,
       -5.4, -5.5, -5.6, -5.7, -5.8, -5.9, -6. ])

In [7]:
p_values_df_W

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
V1,1,0.044979,0.335019,0.349646,0.0,0.690562,0.09321,0.900758,0.43716,0.495825,0.159928,0.286977
V2,0.044979,1,0.143504,0.06275,0.000001,0.001977,0.000919,0.129614,0.46598,0.286977,0.634906,0.001186
V3,0.335019,0.143504,1,0.648644,0.0,0.448563,0.09321,0.859145,0.43716,0.511122,0.157096,0.168656
hV4,0.349646,0.06275,0.648644,1,0.003332,0.112383,0.130832,0.954103,0.398588,0.337429,0.12483,0.34718
VO-1,0.0,0.000001,0.0,0.003332,1,0.043963,0.008108,0.026373,0.003436,0.004191,0.001206,0.085094
VO-2,0.690562,0.001977,0.448563,0.112383,0.043963,1,0.352123,0.648644,0.128405,0.200331,0.066213,0.726242
PHC-1,0.09321,0.000919,0.09321,0.130832,0.008108,0.352123,1,0.170145,0.009465,0.039169,0.007659,0.367215
PHC-2,0.900758,0.129614,0.859145,0.954103,0.026373,0.648644,0.170145,1,0.44284,0.177737,0.000709,0.59114
LO-1,0.43716,0.46598,0.43716,0.398588,0.003436,0.128405,0.009465,0.44284,1,0.564916,0.002213,0.048147
LO-2,0.495825,0.286977,0.511122,0.337429,0.004191,0.200331,0.039169,0.177737,0.564916,1,0.000067,0.111302


In [11]:
arr1 = np.random.rand(80)
arr2 = np.random.rand(80)
res = stats.wilcoxon(arr1, arr2)
res.statistic  
# need to check how to interpret that 

1543.0

In [17]:
# apply rotation on one subject
def apply_rotation(target, subj, rois, sessions):
    # obtain the rotation matrix again
    rotations_df = create_rotation_df(subj, rois, sessions)
    rotations_df = rotations_df[rotations_df['base'] == target]
    indx_to_keep = rotations_df.groupby(['source', 'base'])['distance'].idxmin()
    rotations_df = rotations_df.loc[indx_to_keep]
    rotations_df = rotations_df.reset_index(drop=True)
    print(rotations_df.U)
    rotated_path = os.path.join(proj_dir, 'MDS_rotated', subj)
    print(rotated_path)
    if not os.path.exists(rotated_path):
        os.makedirs(rotated_path)
        print(os.path.exists(rotated_path))
    for roi in rois.keys():
        source_mds = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sessions}_{roi}_mds_betas_train.npy', allow_pickle=False)
        mds_rotated_file = os.path.join(rotated_path, f'{subj}_{sessions}_{roi}_mds_betas_train_rotated_{target}.npy')
        # if the source is our reference point, just save it without applying rotation
        if roi == target:
            if not os.path.exists(mds_rotated_file):
                np.save(mds_rotated_file, source_mds, allow_pickle=True)
            continue 
        U = rotations_df.loc[rotations_df['source'] == roi, 'U'].squeeze()
        mds_rotated = np.dot(source_mds, U)  # apply rotation with dot product
        if not os.path.exists(mds_rotated_file):
            np.save(mds_rotated_file, mds_rotated, allow_pickle=True)
    
    

In [18]:
for i in range(1, 9):
    apply_rotation('PHC-1', f'subj0{i}', rois, sessions[i-1])


0     [[0.3854254, -0.9227389], [0.9227389, 0.3854254]]
1     [[-0.7893548309175643, -0.613937253232856], [0...
2     [[0.98475903, 0.17392436], [-0.17392436, 0.984...
3     [[0.86215484, -0.50664485], [0.50664485, 0.862...
4     [[-0.98772126, -0.15622663], [0.15622663, -0.9...
5     [[-0.6911771960423665, 0.7226853282522155], [-...
6     [[0.65729177, -0.75363624], [0.75363624, 0.657...
7     [[0.7329165, -0.6803186], [0.6803186, 0.7329165]]
8     [[0.13863223248103285, 0.9903439322363339], [-...
9     [[-0.6130681307201219, -0.7900300418941899], [...
10    [[-0.7989624207084763, -0.6013809527210279], [...
Name: U, dtype: object
/home/stan/thesis-repo/projects/NSD/MDS_rotated/subj01
0     [[-0.19842188, -0.9801167], [0.9801167, -0.198...
1     [[0.99857616, -0.053343683], [0.053343683, 0.9...
2     [[0.91326874, 0.40735754], [-0.40735754, 0.913...
3     [[0.9951258, 0.09861478], [-0.09861478, 0.9951...
4     [[-0.610746865004843, 0.7918259069307825], [-0...
5     [[-0.87343055, 0.486

In [19]:
# for all of rotated MDS: try to center them? Find the highest value from 0 